In [4]:
import pandas as pd

df = pd.read_csv(r"D:\Projects\iscale_projects\job_recommender\Data\UpdatedResumeDataSet.csv")


In [5]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [6]:
df.isnull().sum()

Category    0
Resume      0
dtype: int64

In [7]:
df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [8]:
df.columns = ['resume_text', 'job_category']


In [15]:
import pandas as pd

# Load dataset
df = pd.read_csv(r"D:\Projects\iscale_projects\job_recommender\Data\UpdatedResumeDataSet.csv", header=None)  # assuming no headers

# Rename columns for clarity
df.columns = ['Category', 'Resume']

# Function to clean text
def clean_resume(text):
    if pd.isnull(text):
        return ""
    # Decode encoding issues (like â¢)
    try:
        text = text.encode('latin1').decode('utf-8')
    except:
        pass
    # Replace encoded newline characters
    text = text.replace("\\r\\n", "\n").replace("\r\n", "\n")
    # Remove non-ASCII characters
    text = ''.join([c if ord(c) < 128 else ' ' for c in text])
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Apply cleaning
df['Cleaned_Resume'] = df['Resume'].apply(clean_resume)

# Preview cleaned data
print(df[['Category', 'Cleaned_Resume']].head(5))


       Category                                     Cleaned_Resume
0      Category                                             Resume
1  Data Science  Skills * Programming Languages: Python (pandas...
2  Data Science  Education Details May 2013 to May 2017 B.E UIT...
3  Data Science  Areas of Interest Deep Learning, Control Syste...
4  Data Science  Skills R Python SAP HANA Tableau SAP HANA SQL ...


In [16]:
# Save to a new CSV file
df.to_csv("cleaned_resumes.csv", index=False)


In [39]:
df = pd.read_csv(r"D:\Projects\iscale_projects\job_recommender\notebooks\cleaned_resumes.csv",header=1)

In [40]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details May 2013 to May 2017 B.E UIT...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,Data Science,"Education Details MCA YMCAUST, Faridabad, Hary..."


In [32]:
print(df.columns.tolist())


['Category', 'Resume']


In [41]:
import pandas as pd
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

# === Load and clean data ===
df = pd.read_csv(r"D:\Projects\iscale_projects\job_recommender\notebooks\cleaned_resumes.csv",header=1)

def clean_text(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r'\\r\\n', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # remove non-ASCII
    return ' '.join(text.split())

df['cleaned_resume'] = df['Resume'].apply(clean_text)
df['job_category'] = df['Category'].str.strip().str.lower()

# === TF-IDF Vectorization ===
tfidf = TfidfVectorizer(stop_words='english', max_features=3000)
tfidf_vecs = tfidf.fit_transform(df['cleaned_resume'])

# Save TF-IDF vectorizer
with open(r'D:\Projects\iscale_projects\job_recommender\models\tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# === KMeans Clustering ===
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(tfidf_vecs)

# Save KMeans model
with open(r'D:\Projects\iscale_projects\job_recommender\models\kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

# === Train Classifier ===
X = pd.DataFrame(tfidf_vecs.toarray())
X['cluster'] = clusters
X.columns = X.columns.astype(str)  # ensure column names are all strings
y = df['job_category']

clf = RandomForestClassifier(random_state=42)
clf.fit(X, y)

# Save classifier
with open(r'D:\Projects\iscale_projects\job_recommender\models\classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

# Optional: Save cleaned DataFrame
df.to_csv(r'D:\Projects\iscale_projects\job_recommender\Data\cleaned_resumes.csv', index=False)

print("✅ All models and cleaned data saved successfully.")


✅ All models and cleaned data saved successfully.


In [38]:
df.head()

,Category,Cleaned_Resume
0,Category,Resume
1,Data Science,Skills * Programming Languages: Python (pandas...
2,Data Science,Education Details May 2013 to May 2017 B.E UIT...
3,Data Science,"Areas of Interest Deep Learning, Control Syste..."
4,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...


In [42]:
print(df['job_category'].value_counts())


job_category
java developer               84
testing                      70
devops engineer              55
python developer             48
web designing                45
hr                           44
hadoop                       42
sales                        40
data science                 40
mechanical engineer          40
etl developer                40
blockchain                   40
operations manager           40
arts                         36
database                     33
health and fitness           30
pmo                          30
electrical engineering       30
business analyst             28
dotnet developer             28
automation testing           26
network security engineer    25
civil engineer               24
sap developer                24
advocate                     20
Name: count, dtype: int64


In [43]:
df.isnull().sum()

Category          0
Resume            0
cleaned_resume    0
job_category      0
dtype: int64

In [44]:
df.describe()

,Category,Resume,cleaned_resume,job_category
count,962,962,962,962
unique,25,166,166,25
top,Java Developer,"Technical Skills Web Technologies: Angular JS,...","Technical Skills Web Technologies: Angular JS,...",java developer
freq,84,18,18,84


In [45]:
df.head()

,Category,Resume,cleaned_resume,job_category
0,Data Science,Skills * Programming Languages: Python (pandas...,Skills * Programming Languages: Python (pandas...,data science
1,Data Science,Education Details May 2013 to May 2017 B.E UIT...,Education Details May 2013 to May 2017 B.E UIT...,data science
2,Data Science,"Areas of Interest Deep Learning, Control Syste...","Areas of Interest Deep Learning, Control Syste...",data science
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...,Skills R Python SAP HANA Tableau SAP HANA SQL ...,data science
4,Data Science,"Education Details MCA YMCAUST, Faridabad, Hary...","Education Details MCA YMCAUST, Faridabad, Hary...",data science


In [46]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


                           precision    recall  f1-score   support

                 advocate       1.00      1.00      1.00         3
                     arts       1.00      1.00      1.00         6
       automation testing       1.00      1.00      1.00         5
               blockchain       1.00      1.00      1.00         7
         business analyst       0.80      1.00      0.89         4
           civil engineer       1.00      1.00      1.00         9
             data science       1.00      1.00      1.00         5
                 database       1.00      1.00      1.00         8
          devops engineer       1.00      0.93      0.96        14
         dotnet developer       1.00      1.00      1.00         5
   electrical engineering       1.00      1.00      1.00         6
            etl developer       1.00      1.00      1.00         7
                   hadoop       1.00      1.00      1.00         4
       health and fitness       1.00      1.00      1.00     